# <a id='toc1_'></a>[**MICRO-452 Final Report**](#toc0_)
### <a id='toc1_1_1_'></a>[*Group 45: Anton Balykov, Teo Halevi, Cyprien Lacassagne, Seonmin Park*](#toc0_)

# <a id='toc2_'></a>[Table of Contents](#toc0_)


- [Project Objective](#toc0_)    
- [I. Computer Vision](#toc1_)    
  - [I.1 Introduction](#toc1_1_)    
  - [I.2 Camera Set Up](#toc1_2_)    
  - [I.3 Global Obstables and Thymio Detection](#toc1_3_)    
- [II. Global Navigation](#toc2_)    
  - [II.1 Introduction](#toc2_1_)    
  - [II.2 Path Planning](#toc2_2_)    
  - [II.3 Algorithm](#toc2_3_)    
- [III. Local Navigation](#toc3_)    
  - [III.1 Introduction](#toc3_1_)    
  - [III.1 Local Avoidance](#toc3_2_)    
- [IV. Filtering](#toc4_)    
  - [IV.1 Introduction](#toc4_1_)    
  - [IV.2 Methodology](#toc4_2_)    
  - [IV.3 Dynamics of Thymio](#toc4_3_)    
  - [IV.4 Design and Implementation of Filter](#toc4_4_)    
  - [IV.5 Simulation](#toc4_5_)    
- [Conclusion](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc0_'></a>[Project Objective](#toc0_)

This project leverages the Thymio robot to explore and integrate key concepts introduced in the Introduction to Mobile Robotics course. We hereby focus on implementing and demonstrating core components: computer vision, global navigation, motion control, local navigation, and filtering. The mission of the robot is to navigate itself from the designated start position to the specified goal position, successfully avoiding fixed obstacles as well as dynamically placed one. Achieving this requires precise motion control, robust obstable detection, and efficient filtering technique. Such practical application of the five above-mentioned elements would reflect our team's comprehensive understanding of mobile robotics principles.

# <a id='toc1_'></a>[I. Computer Vision](#toc1_)

## <a id='toc1_1_'></a>[I.1 Introduction](#toc1_1_)

Computer vision is the first step in our project, as it is the basic input. To steer the robot in an environment, we need to know the robot's position in that environment, what obstacles are present, where to go, and whether data acquisition is proceeding correctly. This is the purpose of the `ComputerVision` class we have implemented. All this data will then be used to predict the optimal path from the starting position to the goal, avoiding obstacles, and to provide an initial estimate of the position to be given to the Kalmann filter.

We distinguish several parts in this explanation: the general setup, including the camera, the creation of the map, and the creation of markers to detect the robot and the goal. We then talk about detecting obstacles in the map, which the robot must avoid. Finally, the detection of the robot and the goal is a crucial part, which will be done using ArUco markers. 

## <a id='toc1_2_'></a>[I.2 Setup : camera, map, and markers](#toc1_2_)

The first thing to create is the environment in which our robot will evolve. This environment is made up of delimited obstacles in a free zone for the robot. After several computer vision tests, we decided that our obstacles would be represented by red shapes on a white background. This choice is explained in greater detail in the following section on obstacle detection. Once our map has been created, we need to find a way of finding the robot in the image. To do this, we chose to use the ArUco markers provided by the OpenCV library in Python. These markers are QR-code-like representations with unique identifiers that can be easily detected in space. By defining a marker for the robot, and a marker for the goal, we can extract their respective positions to define the robot's shortest path to the goal.

ArUco markers are defined by two things: a dictionary and an identifier. In our case, we use the dictionary `DICT_6X6_250` and IDs 4 and 8 for the robot and goal respectively. A marker is generated as follows, where `marker_id` is the defined ID of the marker, and `marker_size` is the size of the marker in pixels:
```python
dictionary   = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
marker_image = cv2.aruco.generateImageMarker(dictionary, marker_id, marker_size)
```

To have enough markers to test the robustness of our code, we created and printed a grid of 9 markers, for IDs from 0 to 8.

<img src="report-images/aruco_grid.png" alt="aruco_grid" width="500" class="center"/>

The webcam used for our project has a resolution of 1920 by 1080 pixels. The camera is positioned above and in the center of the card, to minimize image distortion. An example of the setup with map, obstacles, robot, robot and goal markers and camera is shown below.

<img src="report-images/original_frame_from_webcam.jpg" alt="Original frame from the webcam" width="750" class="center"/>

When using a camera, two parameters need to be determined: the camera matrix $C$ and the distortion coefficients $D$. Matrix $C$ contains the camera's intrinsic parameters: focal lengths and optical center, along the $x$ and $y$ axes. These parameters are  denoted by: $f_x$, $f_y$ (focal lengths) and $c_x$, $c_y$ (optical center), which gives us: $$C = \begin{pmatrix} f_x & 0 & c_x \\ 0 & f_y & c_y \\ 0 & 0 & 1 \end{pmatrix}$$

The distorsion matrix $D$ is defined as $D = \begin{pmatrix} k_1 & k_2 & p_1 & p_2 & k_3 \end{pmatrix}$ where the parameters are defined in [Camera Calibration](https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html).

Thw two matrices $C$ and $D$ are found using calibration. This step is usually performed using a reference image such as a checkerboard. We used a checkerboard with 9 by 7 inner corners, and with $20$mm square length. By taking several photos from different angles and heights of the checkerboard, we can correct the distorted image. An image of the calibration process is shown below.

<img src="report-images/checkerboard_calibration.png" alt="checkerboard_calibration." width="750" class="center"/>

For our purposes, we have decided to set the origin at the top left corner of the image. To achieve this, the optical centers $c_x$ and $c_y$ were set to $0$. In addition, distortion coefficients were considered negligible when comparing data with and without calibration. Once these matrices are known, the camera is initialized and can be used to detect obstacles and ArUco markers.

## <a id='toc1_3_'></a>[I.3 Detection of global obstacles](#toc1_3)

To be able to steer the robot along the shortest path, we need to know its environment. This means knowing the location of global obstacles on the map.
After several tests, we decided to create our obstacles using flat red shapes. This decision is part of our global obstacle detection strategy.

To detect obstacles in the image, the first step is to apply a mask to the image to extract only the red areas. To do this, we work in the HSV domain of the image. We have determined that the different shades of red can belong to two intervals defined by:

In [ ]:
lower_red_1 = np.array([0, 100, 100])
upper_red_1 = np.array([10, 255, 255])

lower_red_2 = np.array([170, 100, 100])
upper_red_2 = np.array([180, 255, 255])

Our function `apply_color_mask(frame, mask_color='n')` creates the image's red masks for each of the two intervals using the `cv2.inRange` verification function. To bring the masks together, we compare the images pixel by pixel using the OR operator. This returns an image with values of $255$ (white) for pixels detected as red, and $0$ (black) everywhere else.  Finally, the mask undergoes the morphological operation `cv2.MORPH_CLOSE` to reunite disjointed red areas or complete black pixels present in white areas.

The result of this red mask is shown below. We recall the original image. Note that only the red shapes are shown in the black and white image.

<img src="report-images/original_frame_from_webcam.jpg" alt="original_frame" width="500" class="center"/>

<img src="report-images/red_masked_frame.jpg" alt="red_masked_frame" width="500" class="center"/>

This implementation allows us to assume that all obstacles in the image are red. This will facilitate their detection and enable ideal robot 
control.

Once the red shapes have been extracted, we are interested in extracting their edges. Indeed, this is the only important information about an obstacle, since the obstacle's edge must not be exceeded. To extract the edges, we use a Canny edge detector implemented in OpenCV:
```python
cv2.Canny(red_mask, 50, 150)
```

<img src="report-images/red_edges_frame.jpg" alt="red_edges_frame" width="500" class="center"/>

The Canny edge detector involves several stages: noise reduction using Gaussian blurring, gradient calculation, non-maximum suppression, and edge tracking by hysteresis. The second and third arguments in `cv2.Canny` represent the lower and upper thresholds for the hysteresis step. Pixels with gradient intensities above the upper threshold are classified as edges, while those below the lower threshold are ignored. Pixels with intensities between the thresholds are included as edges only if they are connected to a strong edge. The limit values found were obtained after several tests to extract edges on different shades of red, and appear to be optimal for our application.

To predict the shortest path between the robot and the goal, we use a visibility graph. To use it, we need to know the corners of each obstacle. Our function `get_corners_and_shape_edges(edges)` use a polygonal approximation of the shapes found. This approximation allows each obstacle to be represented as a polygon, and therefore to have the corners of these polygons. To study each contour separately, we use `cv2.findContours`. 
```python
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
```

Each contour is then approximated as a polygon, with a certain tolerance. Since each shape is approximated by polygons, rounded shapes can also be defined as a list of corners. This is the case for the ellipse in the figures below.

In [ ]:
# Approximate the contour to a polygon
epsilon = 0.02 * cv2.arcLength(contour, True)
approx  = cv2.approxPolyDP(contour, epsilon, True)

# Extract corner point from the approximated polygon
corners = [[int(point[0][0]), int(point[0][1])] for point in approx]

Finally, the list of corners returned by our function will have the form:
```python
[[[a1,b1], [a2, b2], [a3, b3]],                      # x and y coordinate of the first obstacle
 [[x1, y1], [x2, y2], [x3, y3], [x4, y4], [x5, y5]], # x and y coordinate of the second obstacle
 [[o1, p1], [o2, p2], [o3, p3], [o4, p4]]]           # x and y coordinate of the third obstacle
```

The display of detected contours and corners gives the following image.

<img src="report-images/red_corners_obstacles_frame.jpg" alt="red_corners_obstacles_frame" width="500" class="center"/>

And by displaying the corners of the red obstacles in green on the original camera image, we obtain the following visualization:

<img src="report-images/original_frame_with_corners_obstacles.jpg" alt="original_frame_with_corners_obstacles" width="500" class="center"/>

Note that the above functions give the positions of obstacle corners in pixels. These positions will be converted to millimetres to maintain consistency in the project's units. This conversion is performed using ArUco markers.

## <a id='toc1_4_'></a>[I.4 Detection of the robot and the goal](#toc1_4_)

Global obstacle detection is now implemented. The map in which the robot is moving is therefore known, and obstacles are clearly positioned. The missing step in directing the robot is to know its position, as well as the position of its goal. 

As mentioned in the introduction, we use ArUco markers to identify the robot and the goal. For the following explanations, we'll use the image below with markers 7 (top left), 3 (top right), and 5 (bottom). The photo has been deliberately taken at an angle, with marker 5 blurred to demonstrate the robustness of marker detection.

<img src="report-images/aruco_3markers.jpg" alt="aruco_3markers" width="500" class="center"/>

Basic marker detection is performed by the `cv2.aruco.detectMarkers` function, which takes as arguments the frame to be analyzed, the dictionary used for markers (in our case `DICT_6X6_250`), and initialized parameters. This function returns a list of marker corners, as well as the identifiers of recognized markers.

In the ComputerVision class, the marker detection and marker drawing operation is performed by the following lines, in the function `detect_and_estimate_pose()` and gives the image below:
```python
corners, marker_ids, _ = cv2.aruco.detectMarkers(frame_with_markers, self.aruco_dict, parameters=self.aruco_params)
cv2.aruco.drawDetectedMarkers(frame_with_markers, corners, marker_ids)
```

<img src="report-images/aruco_3markers_with_id.jpg" alt="aruco_3markers_with_id" width="500" class="center"/>

We can see that the markers are detected correctly: they are surrounded by a green border, their ID is shown in blue, and fourth corner of each marker is shown in red.

The positioning of the red corner in the image above gives an indication of the next step in the detection process. The corners of a marker are defined in the following order:
1. Bottom left
2. Bottom right
3. Top right
4. Top left (red corner shown)

where the reference for top, bottom, left, right is given by the conventional reading of the text above the marker.

To extract the position of markers in mm, use the function `cv2.aruco.estimatePoseSingleMarkers` as follows: 
```python
rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners,
                                                      self.ARUCO_MARKER_SIDE_LENGTH,
                                                      self.mtx,
                                                      self.dst)
```

The `self.ARUCO_MARKER_SIDE_LENGTH` parameter defined in class initialization is the length of a marker side in mm. It is used to switch from pixel coordinates to mm coordinates. Note also that the `mtx` and `dst` matrices are used, corresponding respectively to the $C$ and $D$ matrices defined in the calibration explained above.

All this is done by calling the function:
```python
frame_with_markers, marker_ids, rvecs, tvecs, aruco_diagonal_pixels = vision.detect_and_estimate_pose(frame_marker)
```

The result contains two variables `rvecs` (rotation vectors) and `tvecs` (translation vectors).
The rotation vector (`rvecs`) is first converted into a rotation matrix, which provides a representation of 3D orientation. A $4\times 4$ identity matrix is initialized to serve as a transformation matrix, with its top-left $3\times 3$ portion allocated for the rotation information. The function `cv2.Rodrigues()` is then used to convert the rotation vector into a $3\times 3$ rotation matrix.

Next, the $3\times 3$ rotation matrix is converted into a quaternion using the `scipy.spatial.transform.Rotation.from_matrix()` method. A quaternion is a four-dimensional representation $[x, y, z, w]$ of rotation. In the quaternion, $x$, $y$, and $z$ represent the vector part, while $w$ represents the scalar part. Finally, the quaternion is converted to Euler angles using the function `euler_from_quaternion`. This function calculates roll (rotation around the $x$-axis), pitch (rotation around the $y$-axis), and yaw (rotation around the $z$-axis). Among these, the yaw angle is extracted because it represents the rotation of the marker in the $x$-$y$ plane, which corresponds to the marker's orientation relative to the camera.

In [7]:
def process_marker_pose(self, frame, marker_ids, rvecs, tvecs):
    """
    Process pose information for each detected marker and draw on a copy of the original frame.

    original_frame: The frame to process
    marker_ids, rvecs, tvecs: Detected marker details (IDs, rotation vectors, translation vectors)
    mtx, dst: Camera matrix and distortion coefficients

    Returns:
        frame_with_vectors: A new frame with the vectors drawn
        markers_data: Data of the markers (x-position, y-position, yaw-angle for each marker)
    """

    frame_with_vectors = frame.copy()
    markers_data = []

    for i, marker_id in enumerate(marker_ids):
        transform_translation_x = 1000*(tvecs[i][0][0]) # 1000* to have in millimeters.
        transform_translation_y = 1000*(tvecs[i][0][1])
        transform_translation_z = 1000*tvecs[i][0][2]

        rotation_matrix = np.eye(4)
        rotation_matrix[0:3, 0:3] = cv2.Rodrigues(np.array(rvecs[i][0]))[0]
        r = R.from_matrix(rotation_matrix[0:3, 0:3])
        quat = r.as_quat()

        roll_x, pitch_y, yaw_z = self.euler_from_quaternion(quat[0], quat[1], quat[2], quat[3])

        roll_x = math.degrees(roll_x)
        pitch_y = math.degrees(pitch_y)
        yaw_z = -(math.degrees(yaw_z)-90)

        markers_data.extend([[marker_id[0], transform_translation_x, transform_translation_y, yaw_z]])

        cv2.drawFrameAxes(frame_with_vectors, self.mtx, self.dst, rvecs[i], tvecs[i], 0.05)

    return frame_with_vectors, markers_data


def euler_from_quaternion(x, y, z, w):
    """
    Convert a quaternion into euler angles (roll, pitch, yaw)
    roll is rotation around x in radians (counterclockwise)
    pitch is rotation around y in radians (counterclockwise)
    yaw is rotation around z in radians (counterclockwise)
    """

    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    roll_x = math.atan2(t0, t1)

    t2 = +2.0 * (w * y - z * x)
    t2 = np.clip(t2, -1.0, 1.0)
    pitch_y = math.asin(t2)

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    yaw_z = math.atan2(t3, t4)

    return roll_x, pitch_y, yaw_z

By calling the function `process_marker_pose` defined above, we obtain the frame with the detected vectors, as well as the coordinates of each marker.

```python
frame_with_vectors, markers_data = vision.process_marker_pose(frame_with_markers, marker_ids, rvecs, tvecs)
```

<img src="report-images/aruco_3markers_with_vectors.jpg" alt="aruco_3markers_with_vectors" width="500" class="center"/>

For this image, the coordinates are as follows, with ID, $x$-coordinate (mm), $y$-coordinate (mm) and yaw-angle (degrees) for each marker.

```python
[[5, 166.79759785976782, 139.66100169531774, 97.87032032706453],
 [7, 148.05928528540807, 66.24491386374977, 120.86243771133647],
 [3, 296.46820843077455, 67.33202952024861, -45.05418817033191]]
```

The yaw angle is defined as 0 when the green vector is vertical, and follows the clockwise direction. $x$ increases when the marker goes to the right of the image, and $y$ increases when the marker goes to the bottom of the image.

Returning to the original image with the Thymio, we can now display all the information: obstacles, obstacle corners, robot and goal.

<img src="report-images/original_frame_from_webcam.jpg" alt="original_frame" width="500" class="center"/>

<img src="report-images/frame_aruco_and_corners.jpg" alt="frame_aruco_and_corners" width="500" class="center"/>


**NOTE: About conversion from pixels to mm**

ArUco marker coordinates are given directly in mm by the function `detect_and_estimate_pose()`.
On the other hand, the coordinates of obstacle corners are given in pixels.

To have mm everywhere and ensure consistency, we need to convert the coordinates of the obstacle corners into mm. This is done by finding the pixel-mm conversion factor. To do this, we need a length that is both known in mm and known in pixels. We use the information contained in the ArUco markers. In fact, the corners list used in the `detect_and_estimate_pose()` function contains the pixel coordinates of the corners of each marker. Taking the first marker detected, we can calculate the length of its diagonal in pixels. With the actual length of the diagonal in mm known, the conversion factor is found and used to convert the coordinates of the obstacle corners into mm:

```python
aruco_diagonal_pixels = math.sqrt((corners[0][0][2][1] - corners[0][0][0][1])**2 + (corners[0][0][2][0] - corners[0][0][0][0])**2)
```

## <a id='toc1_5_'></a>[I.5 Detection of kidnapping](#toc1_5_)

The last part for the `ComputerVision` class is to detect whether the camera has been covered, or whether the robot has been kidnapped and repositioned elsewhere. We rely on the detection of the marker assigned to the robot to create the corresponding function. If the marker assigned to the robot does not belong to the list of detected markers, then the function returns `True` (to say that the robot has been kidnapped).

In [ ]:
def is_camera_covered(self, detected_markers):
    """
    Returns True (kidnapped) if the ARUCO_ROBOT_ID is not detected, False otherwise
    """

    if detected_markers is None:
        return True
    else:
        return self.ARUCO_ROBOT_ID not in detected_markers

# <a id='toc2_'></a>[II. Global Navigation](#toc2_)

## <a id='toc2_1_'></a>[II.1 Introduction](#toc2_1_)

One main component of this project is navigation. Here, we discuss about our implementation of the global navigation, whose main objective is to find a collision-free path from a source to a goal, on a map filled with polygons acting as obtacles. The environment of motion is imply a 2D flat surface such as the ground or a table. The global, and thus **persistant** obstacles are red polygons made of paper, and placed freely by hand to test the robustness of the path planning algorithm. The boundaries of our map are determined by the field of view - and thus the height - of the camera from the ground, as regions that lie outside the visible field are not processed. To find the optimal path from the robot to the goal, the strategy relies on a metric map and visibility graph, as well as Dijkstra's graph search algorithm.

Global navigation is only involved in the beginning of the program, outside of the main loop. The only case where global path is updated is when the robot is "kidnapped", i.e. moved by hand to another position on the map.

## <a id='toc2_2_'></a>[II.2 Initial Strategy](#toc2_2_)

Initially, we decided to go with a discretized fixed-size cell-map and use Dijktra's search algorithm. Although the implementation was not difficult, we thought that it was not as elegant as the more intuitive visibility graph approach. Nonetheless, here is the result of this preliminary work:

<img src="report-images/cell_decomposition.png" alt="Cell_decomposition" width="500"/>

## <a id='toc2_3_'></a>[II.3 Polygon Expansion](#toc2_3_)

We decided to clearly separate features detection (`ComputerVision`) and path planning (`Navigation`).
The class `Navigation` is constructed by receiving the position of the robot (source), of the goal, and of the vertices of each obstacle. All of this data is extracted from the camera image by the class `ComputerVision`, as explained in Part I. Each obstacle conists of a list of points in the form `[x, y]`.

In [ ]:
import numpy as np
import pyvisgraph as vg

THYMIO_RADIUS = 70 # mm

class Navigation:
    def __init__(self, obstacles, robot, goal):
        """
        :param obstacles: A list of all the obstacles on the map. An obstacle is a list of 
                            vertices, ordered CCW.
        :param robot: The position of the robot.
        :param goal: The position of the goal.
        """
        self.obstacles = obstacles
        self.obstacles_count = len(obstacles)
        self.extended_obstacles = []
        self.source = robot
        self.goal = goal
        self.path = []

Before delving into graph creation, the polygons need to be expanded in order to account for the size of the robot. Otherwise, each obstacle's vertex would be registered as a node, and thus a potential waypoint in the final path, and would be mapped to the center of the robot. This is not ideal since the robot would bump into the obstacles. The function `augment_obstacles` loops through the polygons and computes two new points for each vertex, expanded once in the direction of each of the edges that make up the vertex, and once in the direction of their normals, by a constant factor `THYMIO_RADIUS`, which is the radius of the robot in mm. That way, we ensure the robot always keeps a safe minimal distance with the obstacles, even when some portion of the resulting path is tengential to a polygon edge.

In [ ]:
@staticmethod
def augment_obtacles(self):
    """
    Augments the countour of the obstacles w.r.t. the radius of the robot.
    """
    for polygon in self.obstacles:
        extended_polygon = []

        count = len(polygon)
        for i in range(count):
            vertex = polygon[i]
            prev = polygon[i-1]
            next = (polygon[i+1] if i < count-1 else polygon[0])

            edge1 = np.subtract(vertex, prev)
            edge2 = np.subtract(next,vertex)

            dir1 = np.array(edge1 / np.linalg.norm(edge1))
            dir2 = np.array(edge2 / np.linalg.norm(edge2))

            perp1 = -np.array([dir1[1], -dir1[0]])
            perp2 = -np.array([dir2[1], -dir2[0]])

            v1 = vertex + THYMIO_RADIUS * dir1 + THYMIO_RADIUS * perp1
            v2 = vertex - THYMIO_RADIUS * dir2 + THYMIO_RADIUS * perp2

            # Preserve the winding of the original polygon
            angle = np.arccos(np.clip(np.dot(-dir1, dir2), -1.0, 1.0))
            if angle <= np.pi / 2.0:
                extended_polygon.append(v1)
                extended_polygon.append(v2)
            else:
                extended_polygon.append(v2)
                extended_polygon.append(v1)

        self.extended_obstacles.append(extended_polygon)

Here is the result of our implemantation of polygon expansion. In this example, we set the vertices and thymio's radius to arbitrary values.

<img src="report-images/Polygon_expansion.png" alt="Polygon_expansion" width="500"/>

Expanding the vertices by two points instead of only one, allows for a smoother bypassing and reduces the total path length, while increasing the accuracy of the Kalman filter.

## <a id='toc2_4_'></a>[II.4 Visibility Graph & Graph Search](#toc2_4)

Then, a visibility graph is built using the open-source library [pyvisgraph](https://github.com/TaipanRex/pyvisgraph). This library uses [Lee's algorithm](https://dav.ee/papers/Visibility_Graph_Algorithm.pdf), O(n²log(n)), to compute the visibility of each node.<br>
This operation will fail if the winding of the obstacles is incoherent. That is why we take care of expanding the corners of the obstacles in the right order.

Finally, the shortest path is computed using Dijkstra's algorithm, and returned to the main function.

In [ ]:
def get_shortest_path(self):
    """
    Computes the shortest path from source (robot) to goal, while avoiding the obstacles.

    :returns:
        A list of the coordinates of each node from robot position to goal.
    """
    self.augment_obtacles(self)

    graph = vg.VisGraph()
    polygons = []
    for obstacle in self.extended_obstacles:
        polygon = []
        for point in obstacle:
            polygon.append(vg.Point(point[0], point[1]))

        polygons.append(polygon)
            
    graph.build(polygons)
    path = graph.shortest_path(vg.Point(self.source[0], self.source[1]), vg.Point(self.goal[0], self.goal[1]))

    for point in path:
        self.path.append(point)
        print(point)

    return self.path

 ## <a id='toc2_5_'></a>[II.5 Results](#toc2_5)

Below are the results of two different configurations that shows how the path changes when the first shortest path (left) is blocked (right). A pass through between two obstacles is choked when at least one vertex of a polygon lies into the expanded version of the other one, as you can see on the right image, where we lowered the middle triangle by 50 units.

<img src="report-images/path_planning_1.png" alt="path_planning_1" width="500"/> <img src="report-images/path_planning_2.png" alt="path_planning_2" width="500"/>

# <a id='toc6_'></a>[III. Motion Control & Local Navigation](#toc0_)

## <a id='toc6_1_'></a>[III.1 Introduction](#toc0_)

After extracting the information from the camera and defining the objective for the robot to go to, the next step is to apply motion control and local navigation to guide the robot to the goal.


## <a id='toc6_2_'></a>[III.2 Thymio Class](#toc0_)

To read the readings from Thymio's sensors and control the robot's movement, the `Thymio` class was implemented.

Main tasks of this class are:
- Communicate with the robot via `ClientAsync` from [`tdmclient`](https://pypi.org/project/tdmclient/) library
- Read the sensor values (horizontal proximity and wheels speed)
- Control the robot's movement (set the speed of the wheels)

The class consists of multiple constants and functions to achieve the above tasks.

---
### Constants
1. Constants to store strings of the sensor names and motor names:
    ```
        SENSORS_HORIZONTAL = "prox.horizontal"
        SPEED_LEFT = "motor.left.speed"
        SPEED_RIGHT = "motor.right.speed"

        MOTOR_LEFT = "motor.left.target"
        MOTOR_RIGHT = "motor.right.target"
    ```

2. Constants to set the sensitivity of obstacle detection, reaching the goal and when to start turning towards the next target:
    ```
        GOAL_THRESHOLD = 10
        OBSTACLE_THRESHOLD = 1800
        ANGLE_THRESHOLD = 0.01
    ```

3. `W` matrix with the weights for the horizontal sensors during local navigation:
    ```
        SCALE = 0.03
        W = np.array([[2, 1, -4, -1, -2], [-2, -1, -2, 1, 2]]) * SCALE
    ```

    The visualization of the weights is shown below (**from the Week 3 exercise**):
    
    <img src="./report-images/ANN_robot_control.png" alt="Weights visualization" width="500"/>

    Scaling factor is used to adjust the target speed of the robot.

4. Speed of the robot:
    ```
        SPEED = 70
    ```
    Used to set the speed of the robot's wheels. Was chosen so the robot moves at a reasonable speed enough to turn and not too slow.

5. A constant to account for a sharp turn:
    ```
        SHARP_TURN_THRESHOLD = np.pi / 6
    ```
    As the movement controller was chosen to move in straight lines or turn if there is an angle between the robot's orientation and the target, sometimes the angle and the radius of turn can be too big to reach the next target. THis constant is used in the movement controller to let robot turn in-place instead of more smooth turns. Set to 30 degrees so that robot moves more in straight lines and truns faster.

---

### Functions

1. The instance of the class is created in the `main.py` file to connect to the robot.

In [ ]:
def __init__(self):
    self.client = ClientAsync()
    self.node = aw(self.client.wait_for_node())
    aw(self.node.lock())

    self.goal = None
    self.position = None
    self.orientation = None
    self.is_on_goal = False
    self.is_kidnapped = False

This function also sets the flags needed to define the state of the robot during the simulation.

2. The function to set the motor speeds of the Thymio (used when applying any movement to the robot):

In [ ]:
def set_motors(self, left_motor: int, right_motor: int, verbose: bool = False):
    if verbose:
        print("\t\t Setting speed : ", left_motor, right_motor)

    aw(self.node.set_variables({
        self.MOTOR_LEFT: [left_motor],
        self.MOTOR_RIGHT: [right_motor]
    }))

3. Functions to read sensor data (used to detect obstacles for local navigation and to estimate the robot's position for filtering):

In [ ]:
def get_horizontal_sensors(self, verbose: bool = False):
    aw(self.client.wait_for_status(self.client.NODE_STATUS_READY))
    aw(self.node.wait_for_variables({self.SENSORS_HORIZONTAL}))

    values = self.node.var[self.SENSORS_HORIZONTAL]

    if verbose:
        print("\t\t Sensor values : ", values)

    return np.array(values)[:5]

def get_wheels_speed(self, verbose: bool = False):
    aw(self.client.wait_for_status(self.client.NODE_STATUS_READY))
    aw(self.node.wait_for_variables({self.SPEED_LEFT, self.SPEED_RIGHT}))

    left_speed = self.node.var[self.SPEED_LEFT]
    right_speed = self.node.var[self.SPEED_RIGHT]

    if verbose:
        print("\t\t Wheel speeds : ", left_speed, right_speed)

    return np.array([left_speed[0], right_speed[0]])

4. Setters of the position, orientation and goal of the robot:

In [ ]:
def set_goal(self, goal):
    self.goal = goal

def set_position(self, position: np.ndarray):
    self.position = position

def set_orientation(self, orientation):
    self.orientation = orientation

5. Functions to stop and correctly delete the instance of the class:

In [ ]:
def stop(self):
    self.set_motors(0, 0)

def __del__(self):
    aw(self.node.unlock())

6. Functions to account for kidnapping:

In [ ]:
def kidnap(self):
    self.is_kidnapped = True
    self.stop()

def recover(self):
    self.is_kidnapped = False

7. Function to apply the motion control to the robot:

In [ ]:
def move_to_point(self, target: np.ndarray, verbose: bool = False):
    path = target - self.position

    dst = np.sqrt(np.sum(np.square(path)))

    if dst <= self.GOAL_THRESHOLD:
        if np.sqrt(np.sum(np.square(self.position - self.goal))) <= self.GOAL_THRESHOLD:
            self.is_on_goal = True

        return True

    angle = self.orientation - np.arctan2(path[1], path[0])

    if angle > np.pi:
        angle = angle - 2 * np.pi

    if verbose:
        print("Distance: ", dst)
        print("Orientation: ", self.orientation)
        print("Angle: ", angle)
        print("Position: ", self.position)
        print("Target: ", target)

    if angle > self.ANGLE_THRESHOLD:
        if angle > self.SHARP_TURN_THRESHOLD:
            self.set_motors(self.SPEED, -self.SPEED)
            time.sleep(0.5)
        else:
            self.set_motors(self.SPEED, -self.SPEED)
    elif angle < -self.ANGLE_THRESHOLD:
        if angle < -self.SHARP_TURN_THRESHOLD:
            self.set_motors(-self.SPEED, self.SPEED)
            time.sleep(0.5)
        else:
            self.set_motors(-self.SPEED, self.SPEED)

    self.set_motors(self.SPEED, self.SPEED)

    return False

The `if dst <= self.GOAL_THRESHOLD` check is made in order to check if the target has been reached. There is an additional check for the goal to be reached in case the robot is already on the goal (in that case the `is_on_goal` flag is set to `True`).

Here angle is computed as the difference between the orientation of the robot and the angle of the path to the target. If the angle is greater than the threshold, the robot turns towards the target. Otherwise, it moves forward.
Additional check is made in order to have angles within the range of $[-\pi, \pi]$.
In case of sharp turn, that is defined by the constant `SHARP_TURN_THRESHOLD` (described above), the robot turns in-place (done with the `time.sleep(0.5)` to make sure the robot has turned enough before moving forward).

## <a id='toc6_2_'></a>[III.3 Global Navigation Movement](#toc0_)

Global navigation is implemented via following the points on the global path defined by the path-planning algorithm explained in the previous section.

In [ ]:
# MOVEMENT
target = global_path[0]
if thymio.move_to_point(np.array([target.x * -1, target.y])):
    print(f"Reached checkpoint {check_num}")
    check_num += 1
    global_path.pop(0)

Target is chosen as the next point on the global path. The robot moves towards the target point until it reaches it. Once the target is reached, the point is removed from the global path and the robot moves towards the next point.

*The `target.x * -1` is used to flip the $x$-axis of the target point as the coordinate system of the camera is inverted (the `(0,0)` point is in the top-left corner and the $x$-axis looks to the left compared to the $y$-axis).

<img src="./report-images/coordinate_system.jpg" alt="Coordinate system of the camera" width="500"/>

The inversion was done in order to have the standard Decart coordinate system and to not confuse the robot's movement.

## <a id='toc6_2_'></a>[III.4 Local Navigation Movement](#toc0_)

For the local navigation part the obstacle avoidance algorithm similar to the exercise of **Week 3** is implemented:

In [ ]:
def local_navigation(self):
     while np.any(self.get_horizontal_sensors() > self.OBSTACLE_THRESHOLD):
        motor_values = self.W @ self.get_horizontal_sensors().T
        left_motor = int(motor_values[0])
        right_motor = int(motor_values[1])

        self.set_motors(left_motor, right_motor)

    self.set_motors(self.SPEED, self.SPEED)
    time.sleep(1)

As long as the horizontal sensors of the robot detect an obstacle, the previously described `W` matrix of weights is multiplied by the sensor values and added to the speed of the robot. This way the robot is able to avoid the obstacles by driving around them. The `time.sleep(1)` is used to make sure that it takes some time for the robot to move around the obstacle and only after that continue with the global navigation.

The algorithm is similar to the one mentioned in **Week 3 exercise**:

<img src="./report-images/ANN_algorithm.png" alt="Local Navigation algorithm" width="600"/>

The local navigation routine is handled in the main loop of the program:

In [ ]:
# LOCAL NAVIGATION
if not thymio.is_kidnapped and np.any(thymio.get_horizontal_sensors() > thymio.OBSTACLE_THRESHOLD):
    thymio.local_navigation()

# <a id='toc4_'></a>[IV. Filtering](#toc4_)

## <a id='toc4_1_'></a>[IV.1 Introduction](#toc4_1_)


There is a need for a method that compensates the defect that there is no direct mapping between Thymio's sensors and the values which are used for control. The absolute position of the robot can be obtained directly from the camera and global path planning techniques, but the robot may have low precision and update rate, causing drifts when the velocity or acceleration is integrated. We can address this challenge by implementing Kalman filter as it provides a sound framework to use all available measurements in an optimal way. It relies on a model for the system structure and the effect of disturbances both on the state and on the measurements.

The Kalman filter we implemented estimates the position and speed of the robot when it moves along the planned path. Specifically, we implemented extended Kalman filter to take into account the system involves nonlinear dynamics that would need to be linearized around the current estimate, focusing on local optimality.


## <a id='toc4_2_'></a>[IV.2 Methodology](#toc4_2_)

Prior to the designing of our filter algorithm, we began with estimating Thymio's speed by let it cross seven blocks that are 50 mm each with a sample rate of 0.1 seconds and marked peaks of the ground sensor measurement indicating that it has crossed every block. This was to obtain velocity parameters, in which the method is identical to one of the activities found in Exercise 8 of the course. With known and obtained information, we could obtain the conversion factor of 0.4 to convert Thymio's speed to mm/s.

<img src="report-images/find_peaks.png" alt="Find Peaks" width="500"/>

Now, to obtain speed variance, we observed the data where the robot was moving by plotting the robot's speed and computed average velocity and thus the speed variance by diving the average speed with the robot's speed in mm/s.

<img src="report-images/measured_vel.png" alt="Measured Vel" width="500"/>

With the assumption that half of the variance is cause by the measurement and the other half is caused by the perturbation to the states, we obtain 

$q_v = r_v = 16.08621$ $mm^2/s^2$ $\space$ where $\space$ $\sigma^2_{vel} = 32.17242$ $mm^2/s^2$

Variances on position state and measurement as well as those for the angle for orientation ($\sigma^2_{pos}$ and $\sigma^2_{\theta}$) were an arbitrary value that were tuned manually for the specific Thymio robot that is going to be used for demonstration.

## <a id='toc4_3_'></a>[IV.3 Dynamics of Thymio](#toc4_3_)

To begin with the implementation, the state space model is

$\hat{x}_{k-1|k-1} =  \begin{bmatrix}
                        x_{pos_{k-1}} \\
                        y_{pos_{k-1}} \\
                        \theta_{k-1} \\
                        v_{left_{k-1}} \\
                        v_{right_{k-1}} \\
                        \end{bmatrix}$

Under control input vector,

$ u =[v_{trans}, v_{rot}] $, $\space$ where $\space$ $v_{tran} = \frac{v_{left} + v_{right}}{2} $ $\space$ and $\space$ $ v_{rot} = \frac{v_{left} - v_{right}}{l_{base}} $

<img src="report-images/Thymio_odometry.png" width="500"/>

Then, we can see that the robot's new position and orientation are predicted in the following way

$x_{pos_{k}} = x_{pos_{k-1}} + v_{trans_{k}} \cdot \cos(\theta_{k-1}) \cdot dk$

$y_{pos_{k}} = y_{pos_{k-1}} + v_{trans_{k}} \cdot \sin(\theta_{k-1}) \cdot dk$

$\theta_{k} = \theta_{k-1} + v_{rot_{k}} \cdot dk$

## <a id='toc4_4_'></a>[IV.4 Design and Implementation of Filter](#toc4_4_)



Before diving into the steps of the extended Kalman filter we implemented, we would like to disclaim that the notations and formulas were referenced from lecture slides 8 of the Mobile Robotics course and online tutorial at [Extended Kalman Filter (EKF) with Python Code Example](https://automaticaddison.com/extended-kalman-filter-ekf-with-python-code-example/).

We have introduced that the extended Kalman filter takes into account nonlinear system dynamics in the previous sections. In order to make prediction of the next state, We use a nonlinear motion model $f(x, u)$. That is, we can formulate their relation as:

$\hat{x}_{k|k-1} = f(\hat{x}_{k-1|k-1}, u_k)$

Our $f(x, u)$ is then introduced as

$f(\hat{x}_{k-1|k-1}, u_k) = \begin{bmatrix}
1 & 0 & 0 & 0.5 \cdot \cos(\theta_{k-1}) \cdot dk & 0.5 \cdot \cos(\theta_{k-1}) \cdot dk \\ 
0 & 1 & 0 & 0.5 \cdot \sin(\theta_{k-1}) \cdot dk & 0.5 \cdot \sin(\theta_{k-1}) \cdot dk \\ 
0 & 0 & 1 & -dk/l_{base} & dk/l_{base} \\ 
0 & 0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 0 & 1 
\end{bmatrix}$

In this prediction step of the filter, we make a use of the predicted covariance matrix $P_{k-1|k-1}$.

$P_{k-1|k-1} = \begin{bmatrix}
\sigma^2_{pos} & 0 & 0 & 0 & 0 \\ 
0 & \sigma^2_{pos} & 0 & 0 & 0 \\ 
0 & 0 & \sigma^2_{\theta} & 0 & 0 \\ 
0 & 0 & 0 & \sigma^2_{vel} & 0 \\ 
0 & 0 & 0 & 0 & \sigma^2_{vel} 
\end{bmatrix}$

as each column corresponds to the state variables, from the first column to the last column, being $x_{pos_{k-1}}$, $y_{pos_{k_1}}$, $\theta_{k-1} $, $v_{left_{k-1}}$, and $v_{right_{k-1}}$, respectively.

The updated (current) covariance matrix is as follows:

$P_{k|k-1} = F_{k}P_{k-1|k-1}F_{k}^{T} + Q_{k}$

where $F_{k}$ is the Jacobian of $f_{k}$ matrix, and $Q_{k}$ is the noise covariance matrix of the state model. The $Q_{k}$ matrix specifically represents the degree or the extent of how much the actual motion of Thymio deviates from its assumed state space model which would be the path that it should follow for our case.

We can write the two matrices as below:

$F_{k|k-1} = 
\begin{bmatrix}
1 & 0 & -v_{trans} \cdot \sin(\theta_{k-1}) \cdot dk & 0.5 \cdot \cos(\theta_{k-1}) \cdot dk & 0.5 \cdot \cos(\theta_{k-1}) \cdot dk \\ 
0 & 1 & v_{trans} \cdot \cos(\theta_{k-1}) \cdot dk & 0.5 \cdot \sin(\theta_{k-1}) \cdot dk & 0.5 \cdot \sin(\theta_{k-1}) \cdot dk \\ 
0 & 0 & 1 & -dk/l_{base} & dk/l_{base} \\ 
0 & 0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 0 & 1 
\end{bmatrix}$

$Q_{k|k-1} = \alpha \cdot 
\begin{bmatrix}
\sigma^2_{pos} & 0 & 0 & 0 & 0 \\ 
0 & \sigma^2_{pos} & 0 & 0 & 0 \\ 
0 & 0 & \sigma^2_{\theta} & 0 & 0 \\ 
0 & 0 & 0 & \sigma^2_{vel} & 0 \\ 
0 & 0 & 0 & 0 & \sigma^2_{vel} 
\end{bmatrix}$

where $\alpha$ is a scaling factor, which we have initialized as $\alpha =10$, such that 

$Q_{k-1|k-1} = 10 \cdot 
\begin{bmatrix}
\sigma^2_{pos} & 0 & 0 & 0 & 0 \\ 
0 & \sigma^2_{pos} & 0 & 0 & 0 \\ 
0 & 0 & \sigma^2_{\theta} & 0 & 0 \\ 
0 & 0 & 0 & \sigma^2_{vel} & 0 \\ 
0 & 0 & 0 & 0 & \sigma^2_{vel} 
\end{bmatrix}$

We set $\alpha$ large enough so that the filter tracks large changes in the wheel sensor measurements more closely as if it was smaller.

In the updating process of the extended Kalman filter, we make a use of the term *innovation* which is the difference between the actual measurements and predicted observations. Let $z_{k}$ be the observation vector and $i_{k}$ be the innovation term. Then, we can compute:

$i_{k} = z_{k} - H_{k}\hat{x}_{k-1}$ 

where $H_{k}$ is the measurement matrix that is used to convert the predicted state estimate at time $k$ into predicted sensor measurements at time $k$.

In this project, we have two cases that can happen. One of them is when Thymio gets information from the camera, we name it as "when it has vision," and the other is when it does not get information from the camera, "when it has no vision," specifically when the camera is covered.

When it has vision, 

$H_{k} = \begin{bmatrix}
1 & 0 & 0 & 0 & 0 \\ 
0 & 1 & 0 & 0 & 0 \\ 
0 & 0 & 1 & 0 & 0 \\ 
0 & 0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 0 & 1 
\end{bmatrix}$

When it does not have vision, 

$H_{k} = \begin{bmatrix}
0 & 0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 0 & 1 
\end{bmatrix}$

as the robot will have to rely only on its wheel sensor measurements in this scenario.

With the innovation $y_{k}$, we obtain the innovation covariance matrix $S_{k}$ to be used for the computation of Kalman Gain $K_{k}$.

$S_{k} = H_{k}P_{k|k-1}H_{k}^{T} + R_{k}$

where $R_{k}$ is the sensor measurement noise covariance matrix. It also has different dimensionality depending on if Thymio has vision or not.

When it has vision,

$R_{k} = \beta \cdot \begin{bmatrix}
\sigma^2_{pos} & 0 & 0 & 0 & 0 \\ 
0 & \sigma^2_{pos} & 0 & 0 & 0 \\ 
0 & 0 & \sigma^2_{\theta} & 0 & 0 \\ 
0 & 0 & 0 & \sigma^2_{vel} & 0 \\ 
0 & 0 & 0 & 0 & \sigma^2_{vel} 
\end{bmatrix}$

When it does not have vision,

$R_{k} = \gamma \cdot \begin{bmatrix}
\sigma^2_{vel} & 0\\ 
0 & \sigma^2_{vel} 
\end{bmatrix}$

with the same logic of $H_{k}$ matrices, and constants $\beta = 1$ and $\gamma = 1$ were defined through trial and error experiments just like $\alpha$ of the $Q_{k}$ matrix.

The Kalman Gain matrix is computed as follows:

$K_{k} = P_{k|k-1}H_{k}^{T}S_{k}^{-1}$

and we can update our estimated state and its covariance matrix by

$\hat{x}_{k|k} = \hat{x}_{k|k-1} + K_{k}i_{k}$ $\space$ and $\space$ $ P_{k|k} = (I - K_{k}H_{k})P_{k|k-1}$ $\space$,  respectively.


The above steps are implemented as a class ***KalmanFilterExtended*** and the class is called in the **main.py** file where all the five components of the project are integrated.

The function ***run_ekf()*** ensures that the extened filter is intiated and the snippet from **main.py** below ensures that the filter is running in a loop throughout the entire mission of the robot.

Additionally, we have set parameters that defines *kidnapping* in terms of the difference in the Thymio's orientation and position exceeding the threshold values. Specifically, we have set the threshold for orientation and position to be 35cm and 60°, and exceeding either (or both) of these values would be considered getting kidnaped.

In [ ]:
def prediction(self, dt):
        f, F = self.compute_fnF(dt)
        self.x = f @ self.x
        self.P = F @ self.P @ F.T + self.Q

In the updating process of the extended Kalman filter, we make a use of the term *innovation* which is the difference between the actual measurements and predicted observations. Let $z_{k}$ be the observation vector and $i_{k}$ be the innovation term. Then, we can compute:

$i_{k} = z_{k} - H_{k}\hat{x}_{k-1}$ 

where $H_{k}$ is the measurement matrix that is used to convert the predicted state estimate at time $k$ into predicted sensor measurements at time $k$.

In this project, we have two cases that can happen. One of them is when Thymio gets information from the camera, we name it as "when it has vision," and the other is when it does not get information from the camera, "when it has no vision," specifically when the camera is covered.

When it has vision, 

$H_{k} = \begin{bmatrix}
1 & 0 & 0 & 0 & 0 \\ 
0 & 1 & 0 & 0 & 0 \\ 
0 & 0 & 1 & 0 & 0 \\ 
0 & 0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 0 & 1 
\end{bmatrix}$

When it does not have vision, 

$H_{k} = \begin{bmatrix}
0 & 0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 0 & 1 
\end{bmatrix}$

as the robot will have to rely only on its wheel sensor measurements in this scenario.

With the innovation $y_{k}$, we obtain the innovation covariance matrix $S_{k}$ to be used for the computation of Kalman Gain $K_{k}$.

$S_{k} = H_{k}P_{k|k-1}H_{k}^{T} + R_{k}$

where $R_{k}$ is the sensor measurement noise covariance matrix. It also has different dimensionality depending on if Thymio has vision or not.

When it has vision,

$R_{k} = \beta \cdot \begin{bmatrix}
\sigma^2_{pos} & 0 & 0 & 0 & 0 \\ 
0 & \sigma^2_{pos} & 0 & 0 & 0 \\ 
0 & 0 & \sigma^2_{\theta} & 0 & 0 \\ 
0 & 0 & 0 & \sigma^2_{vel} & 0 \\ 
0 & 0 & 0 & 0 & \sigma^2_{vel} 
\end{bmatrix}$

When it does not have vision,

$R_{k} = \gamma \cdot \begin{bmatrix}
\sigma^2_{vel} & 0\\ 
0 & \sigma^2_{vel} 
\end{bmatrix}$

with the same logic of $H_{k}$ matrices, and constants $\beta = 1$ and $\gamma = 1$ were defined through trial and error experiments just like $\alpha$ of the $Q_{k}$ matrix.

The Kalman Gain matrix is computed as follows:

$K_{k} = P_{k|k-1}H_{k}^{T}S_{k}^{-1}$

and we can update our estimated state and its covariance matrix by

$\hat{x}_{k|k} = \hat{x}_{k|k-1} + K_{k}i_{k}$ $\space$ and $\space$ $ P_{k|k} = (I - K_{k}H_{k})P_{k|k-1}$ $\space$,  respectively.


In [ ]:
def update(self, position, u, cam):
        if cam:
            R = self.R_cam
            H = self.H_cam
            z = np.array([position[0], position[1], position[2], u[0], u[1]])
        else:
            R = self.R_nocam
            H = self.H_nocam
            z = np.array([u[0], u[1]])

        z_pred = H @ self.x
        y = z - z_pred

        S = H @ self.P @ H.T + R
        K = self.P @ H.T @ np.linalg.pinv(S)
        self.x += K @ y
        self.P = self.P - K @ H @ self.P

The above steps are implemented as a class `KalmanFilterExtended` and the class is called in the `main.py` file as
```python
    ekf = KalmanFilterExtended(np.array([thymio_pos_x * -1, thymio_pos_y, ((thymio_theta + 180) % 360) * np.pi / 180]), u)
```

The function `run_EKF` ensures that the extened filter is intiated the filter steps are correctly called in the correct order. The code snippet below is part of it.

```python
    ekf.prediction(dt)
    measured_state = ekf.get_state()
    ekf.update([pos_x, pos_y, theta], u, cam)
    measurement_update = ekf.get_state()
``` 

It helps us to have the filter run in `main.py` in a loop throughout the entire mission of the robot as

```python
    measured_state, kidnap = run_EKF(ekf, thymio_pos_x * -1, thymio_pos_y, theta, u, cam=camera)
```

Additionally, we have set parameters that define *kidnapping* in terms of the difference in the Thymio's orientation and position exceeding the threshold values. This mode is apart from the *kidnapping* condition from the Computer Vision side, and we decided to have two modes of kidnap detection to account for a variety of possible ways to kidnap the robot. Specifically, we have set the threshold for orientation and position to be 100 mm and 1 rad, and exceeding either (or both) of these values would be considered getting kidnaped.

```python
    if dpos > 100 or dtheta > 1:
        kidnap = True
        print("Thymio is being kidnapped (moved too far or rotated too much)")
        cur_t = time.time()
        ekf.count_time(cur_t)
        ekf.get_state()
    else:
        kidnap = False
``` 

We have also added a `draw_confidence_ellipse()` function that draws confidence ellipse taking from the first two diagonal components of the $P_{k}$ matrix as a means to visualize the uncertainty that the system has regarding the robot's pose.
```python
    P = get_cov().[:2, :2]
```

The method of implementing it was referenced from Github tutorial on [Robot Motion](https://github.com/jotaraul/jupyter-notebooks-for-robotics-courses/blob/master/3-Robot%20motion.ipynb) and [The Kalman Filter](https://engineeringmedia.com/controlblog/the-kalman-filter)

Then the extracted $P_{k}$ matrix will be

$P_{k} = \begin{bmatrix}
\sigma_{xx}^2 & \sigma_{xy}^2 \\ 
\sigma_{yx}^2 & \sigma_{yy}^2 \\
\end{bmatrix}$

We computed the eigenvalues and eigenvectors of it as the major and minor axis of the confidence ellipse is determined by the eigenvalues and the angles of the confidence ellipse is determined by the eigenvectors.

```python
    eigenvalues, eigenvectors = np.linalg.eigh(P)
    angle = np.degrees(np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0]))
``` 

Then, we used ChatGPT to add chi-squared confidence interval to the ellipse computing function to account for the nonlinearity of the system.

```python
    chi_squared_val = np.sqrt(5.991) 
    amplification = 1
    major_axis = chi_squared_val * np.sqrt(eigenvalues[0]) * amplification
    minor_axis = chi_squared_val * np.sqrt(eigenvalues[1]) * amplification

    center = (int(mean[0]), int(mean[1]))
    axes = (int(major_axis), int(minor_axis))
    cv2.ellipse(frame, center, axes, angle, 0, 360, color, 2)
``` 
In `main.py`, it is called as
```python
    draw_confidence_ellipse(frame_aruco_and_corners, ekf.get_cov()[:2, :2], [x, y], color=(0, 255, 255))
```
in the loop of updating every measurement so that the ellipse is also plotted dynamically.
This visualization of positional uncertainty in 2D space provides better intuition of how the uncertainty changes during the mission and different modes such as kidnapping and switching between vision and no vision.

## <a id='toc5_5_'></a>[IV.5 Simulation](#toc5_5_)

The simulation on filtering was done several times on the path planning scenario found in the previous section of Global Navigation. One of the examples is shown below, where the `run_EKF` receives a set $x$ and $y$ coordinate of the planned path in each iteration and applies filtering from the `KalmanFilterExtended` class functions. The code snippet of this can be found in `KFsim.py`. The purpose of this simulation is to observe whether the filtering is successfully running throughout the entire travel sequence and to observe different results in two different conditions: when Thymio has vision and when it does not. The simulation is done by imposing random noises of normal distribution on the wheel sensors and camera vision and compare the path the robot would take with the true path the path planning provides. When the simulation confirms that the filtering is operating well, we could integrate the algorithm into our `main.py`.

<img src="report-images/EKF_simulation.png" width="571"/>
<img src="report-images/kalman_simu.png" width="571"/>

# <a id='toc4_'></a>[IV. Main Alogrithm](#toc4_)

Let us now explain how the main algorithm works (whole code can be found in [`main.py`](/main.py)).

To start the algorithm the # of obstacles has to be provided with the command-line argument (this will be used to extract global obstacles):

In [ ]:
NUMBER_OF_OBSTACLES = int(sys.argv[1])

if NUMBER_OF_OBSTACLES < 0:
    print("Please provide a number of obstacles greater than 0")
    os.close(0)

Then the Computer Vision instance is initialized and started. The `while` loop was added in order to avoid noise from the camera when it starts.

In [ ]:
vision = ComputerVision(camera_index=1, robot_id=4, goal_id=8)
corners = []

ret, frame = vision.cam.read()
# While loop to avoid noisy detections when camera starts
while True:
    if not ret:
        print("Failed to capture frame. Exiting.")

    frame_masked = vision.apply_color_mask(frame,  mask_color='r')  # We apply a red mask to detect only red obstacles
    frame_with_vectors, markers_data, marker_ids, conversion_factor = get_frame_with_vectors(vision, frame)
    edges = vision.detect_edges(frame_masked)  # We detect the edges on the masked frame using Canny
    corners = vision.get_corners_and_shape_edges(edges)  # We obtain shapes and corners by approxPolyDP

    ret, frame = vision.cam.read()

    # If all the obstacles are detected, we can break the loop
    if len(corners) == NUMBER_OF_OBSTACLES:
        break

# Conversion from pixels to mm
corners_mm = [np.array(shape) / conversion_factor for shape in corners]

Here the mask is applied, the edges and corners are then extracted from the image. The conversion factor is used to convert the pixel values to mm. The loop is broken when all the obstacles are detected.

After that, Thymio's position and goal position are extracted from the markers data.

In [ ]:
thymio_pos_x, thymio_pos_y, thymio_theta = get_thymio_localisation(markers_data)
goal_pos = get_goal_position(markers_data)

It is done with the helper functions added in the same file for the sake of clarity.

Global path is then found using the path planning algorithm explained in the previous sections.

In [ ]:
global_goal = deepcopy(goal_pos)
global_obstacles = deepcopy(corners_mm)
navigation = Navigation(global_obstacles, [thymio_pos_x, thymio_pos_y], global_goal)
global_path = navigation.get_shortest_path()

Thymio's position and orientation are set to the initial values got from the markers data.

In [ ]:
thymio = Thymio()
thymio.set_position(np.array([thymio_pos_x * -1, thymio_pos_y]))
thymio.set_orientation(((thymio_theta + 180) % 360) * np.pi / 180)
goal_pos[0] = -goal_pos[0]
thymio.set_goal(goal_pos)

As was already explained in the **Motion Control & Local Navigation** section, the coordinate system is reversed and both the goal and Thymio's positions are flipped on the $x$-axis.
Thymio's orientation is also flipped so it aligns with the correct $x$-axis. It is then transformed to radians.

The Extended Kalman Filter is then initialized with the initial position and velocity.

In [ ]:
u = thymio.get_wheels_speed()
ekf = KalmanFilterExtended(np.array([thymio_pos_x * -1, thymio_pos_y, ((thymio_theta + 180) % 360) * np.pi / 180]), u)

The main loop of the program is then started. The loop is running until the robot reaches the goal.

In [ ]:
while not thymio.is_on_goal:

Local navigation is applied as it was mentioned in the **Motion Control & Local Navigation** section.

In [ ]:
if not thymio.is_kidnapped and np.any(thymio.get_horizontal_sensors() > thymio.OBSTACLE_THRESHOLD):
    thymio.local_navigation()

Edges and corners are detected with the Computer Vision algorithm in the same way as it was done at the beginning of the program. Then the results are drawn on the frame for the visualization.

In [ ]:
draw_obstacles(frame_aruco_and_corners, navigation, conversion_factor)
draw_path(frame_aruco_and_corners, drawing_path, conversion_factor)

Then the check for covered camera is done to detect if we can detect the Thymio and the goal.

In [ ]:
camera_covered = vision.is_camera_covered(marker_ids)
    if camera_covered:
        goal_pos = get_goal_position(markers_data)
        if goal_pos is None:
            print("Camera is covered!")
            camera = False
        else:
            print("Thymio is being kidnapped (marker not detected)")
            thymio.kidnap()
            continue
    else:
        camera = True

If the camera is covered but the goal is still visible it means that the Thymio is kidnapped and the `kidnap()` function is called. The `continue` statement is used to skip the rest of the loop and go to the next iteration until the Thymio is visible again and the new global path can be recalculated.

In [ ]:
if thymio.is_kidnapped:
    thymio.recover()
    navigation = Navigation(global_obstacles, [thymio_pos_x, thymio_pos_y], global_goal)
    global_path = navigation.get_shortest_path()
    drawing_path = deepcopy(global_path)
    check_num = 0

    u = thymio.get_wheels_speed()
    ekf = KalmanFilterExtended(np.array([thymio_pos_x * -1, thymio_pos_y, ((thymio_theta + 180) % 360) * np.pi / 180]), u)

The second figure shows the confidence ellipses, where the purple ellipses represent with vision and the yellow ellipses represent without vision. For visualization purposes, the purple ellipses were amplified 150-fold and the yellow ellipses were amplified 50-fold. Even with higher amplification factor, the confidence ellipses are significantly smaller for with vision case than for the without vision case, which means that the system tend to have much more certainty when the camera is not covered than when the camera is covered. The yellow ellipses grow with time because the longer the robot is blocked from vision, the more uncertain further steps will be.

# <a id='toc6_'></a>[Conclusion](#toc6_)
This project was a valuable opportunity to apply the theory we learned about mobile robotics. We tried to cover and use the concepts of the course as much as we could, from computer vision, to navigation and motion control uing Kalman filtering. 
This kind of project is always a good way to reinforce our ability to work as a group and to use collaborative tools like git.
The split of the tasks made debugging easier until the first integrated runs. 

We faced some challenges:
- Polygon vertices detection
- Completeness of path planning
- Robutness of Kalman filter

But we were able to work together and solve them without too much difficulty.

At the end, although one might say our Thymio is rather slow, it never fails to reach its goal.